In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load IMDB dataset
max_features = 10000  # Vocabulary size
maxlen = 500  # Sequence length

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

# Build the model
from tensorflow.keras.layers import Dropout

model = Sequential([
    Embedding(input_dim=max_features, output_dim=128, input_length=maxlen),
    SimpleRNN(128, activation='relu', return_sequences=False),
    Dropout(0.5),  # Add dropout
    Dense(1, activation='sigmoid')
])


# Build the model explicitly
model.build(input_shape=(None, maxlen))  # Batch size is None for flexibility

from tensorflow.keras.optimizers import Adam

optimizer = Adam(learning_rate=0.0001)  # Reduce learning rate
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])


# Display the summary
model.summary()


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)              │ (None, 500, 128)            │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_4 (SimpleRNN)             │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,313,025 (5.01 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
from tensorflow.keras.callbacks import EarlyStopping
earlystopping=EarlyStopping(monitor='val_loss',patience=2,restore_best_weights=True)
earlystopping

In [14]:
history=model.fit(x_train,y_train,epochs=10,batch_size=32,
                  validation_split=0.2,
                  callbacks=[earlystopping])

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 38s 57ms/step - accuracy: 0.5332 - loss: 0.6906 - val_accuracy: 0.5870 - val_loss: 0.6802
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 39s 56ms/step - accuracy: 0.6539 - loss: 0.6437 - val_accuracy: 0.7002 - val_loss: 0.6027
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 35s 56ms/step - accuracy: 0.7807 - loss: 0.5099 - val_accuracy: 0.8562 - val_loss: 0.3416
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 41s 56ms/step - accuracy: 0.8828 - loss: 0.2950 - val_accuracy: 0.8738 - val_loss: 0.3071
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 41s 56ms/step - accuracy: 0.9259 - loss: 0.2103 - val_accuracy: 0.8660 - val_loss: 0.3186
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 35s 56ms/step - accuracy: 0.9288 - loss: 0.2058 - val_accuracy: 0.8768 - val_loss: 0.3104


In [15]:
## Save model file
model.save('simple_rnn_imdb.h5')